In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 9.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660427 sha256=c1551d173d6cbd6dd267c186a2579ed17f014604169875005202e5c5e239d150
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [17]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time

# CUDA kernel for matrix multiplication (optimized with shared memory)
cuda_code = """
__global__ void matrixMul(float *A, float *B, float *C, int M, int N, int K) {
    const int TILE_SIZE = 16;
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum = 0.0f;
    for (int t = 0; t < (N + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Load tiles into shared memory
        if (row < M && t * TILE_SIZE + threadIdx.x < N)
            As[threadIdx.y][threadIdx.x] = A[row * N + t * TILE_SIZE + threadIdx.x];
        else
            As[threadIdx.y][threadIdx.x] = 0.0f;

        if (t * TILE_SIZE + threadIdx.y < N && col < K)
            Bs[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * K + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();

        // Compute partial sum for the tile
        for (int i = 0; i < TILE_SIZE; i++)
            sum += As[threadIdx.y][i] * Bs[i][threadIdx.x];

        __syncthreads();
    }

    if (row < M && col < K)
        C[row * K + col] = sum;
}
"""

# Function to input a 3x3 matrix row by row
def input_matrix(matrix_name):
    rows, cols = 3, 3
    print(f"Enter elements for matrix {matrix_name} (3x3):")
    matrix = np.zeros((rows, cols), dtype=np.float32)
    for i in range(rows):
        while True:
            try:
                # Prompt for one row
                row_input = input(f"Enter 3 elements for row {i+1} (space-separated): ")
                values = [float(x) for x in row_input.split()]
                if len(values) != cols:
                    print(f"Expected 3 elements, got {len(values)}. Try again.")
                    continue
                matrix[i, :] = values
                break
            except ValueError:
                print("Invalid input. Please enter valid numbers.")
    return matrix

# Fixed matrix dimensions for 3x3 matrices
M, N, K = 3, 3, 3  # A: 3x3, B: 3x3, C: 3x3

# Input matrices A and B
A = input_matrix("A")
B = input_matrix("B")

# Print input matrices for verification
print("\nMatrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Compile the CUDA kernel
mod = SourceModule(cuda_code)
matrix_mul = mod.get_function("matrixMul")

# Allocate GPU memory
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(M * K * np.float32().nbytes)

# Copy input matrices to GPU
cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)

# Initialize C_gpu to zero
C_init = np.zeros((M, K), dtype=np.float32)
cuda.memcpy_htod(C_gpu, C_init)

# Set block and grid dimensions
block_size = (16, 16, 1)
grid_size = ((K + block_size[0] - 1) // block_size[0], (M + block_size[1] - 1) // block_size[1])

# Execute the kernel
start_time = time.time()
matrix_mul(A_gpu, B_gpu, C_gpu,
           np.int32(M), np.int32(N), np.int32(K),
           block=block_size, grid=grid_size)
cuda.Context.synchronize()
gpu_time = time.time() - start_time

# Copy result back to host
C = np.empty((M, K), dtype=np.float32)
cuda.memcpy_dtoh(C, C_gpu)

# Verify result by comparing with NumPy
C_np = np.dot(A, B).astype(np.float32)
if np.allclose(C, C_np, atol=1e-4, rtol=1e-4):
    print(f"\nGPU computation successful! Time taken: {gpu_time:.4f} seconds")
    print("\nResult matrix C (GPU):")
    print(C)
else:
    print("\nGPU computation failed!")
    print(f"Max difference: {np.max(np.abs(C - C_np))}")
    print("\nResult matrix C (GPU):")
    print(C)
    print("\nExpected matrix C (NumPy):")
    print(C_np)

# Free GPU memory
A_gpu.free()
B_gpu.free()
C_gpu.free()

Enter elements for matrix A (3x3):
Enter 3 elements for row 1 (space-separated): 1 2 3
Enter 3 elements for row 2 (space-separated): 1 2 3
Enter 3 elements for row 3 (space-separated): 1 2 3
Enter elements for matrix B (3x3):
Enter 3 elements for row 1 (space-separated): 3 4 5
Enter 3 elements for row 2 (space-separated): 5 6 7
Enter 3 elements for row 3 (space-separated): 8  9 0

Matrix A:
[[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]]

Matrix B:
[[3. 4. 5.]
 [5. 6. 7.]
 [8. 9. 0.]]

GPU computation successful! Time taken: 0.0002 seconds

Result matrix C (GPU):
[[37. 43. 19.]
 [37. 43. 19.]
 [37. 43. 19.]]
